In [1]:
import tweepy
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
!pip freeze | grep tweepy

tweepy==3.7.0


In [2]:
consumer_key = 'wxgWPzoSV1jJ4w9ickrJ7vEbi'
consumer_secret = 'bXX0bMVXyeff3BtlchzBiNyoSxAWNaUfC7r2TlYjKygeK2o6XF'
access_token = '1041110068735832064-ZyCKSxnFVqjbRBf7yQnQQ1ES4LHWoO'
access_token_secret = 'Iap9tp6knHDHkxVhAolscYHQPir0MvPBJyN4lnzm8k2mF'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)  # twitter consumer key and consumer secret
auth.set_access_token(access_token, access_token_secret)  # twitter access token and access_token_secret

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())


In [3]:
def top_word_count(text, min_word_length=2, top=50):
    """
    Get word count out of text
    It also removes all single character words
    :param text: input text
    :param min_word_length: min characters to consider as a word
    :param top: top number of words to be returned
    :return: top words
    """
    wordcount = {}
    for word in text.split():
        if word not in wordcount:
            wordcount[word] = 1
        else:
            wordcount[word] += 1
    final_wordcount = {}
    for key, value in wordcount.items():
        if (len(key) >= min_word_length):
            final_wordcount[key] = value
    final_wordcount = sorted(final_wordcount.items(), key=lambda x: x[1], reverse=True)
    return final_wordcount[:top]


In [4]:
def get_group_summary(group, category):
    """
    Summarize tweeter groups
    :param group:
    :return: summary json
    """
    group['Text'] = group['Text'].str.replace(r'[{}]+'.format(category), '')
    top_words = top_word_count(" ".join(group['Text'].values))
    return {'word_count': top_words,
            'retweet_sum': group['Retweet Count'].sum(),
            'favorite_sum': group['Favorite Count'].sum(),
            'reply_count': group[group['Is Reply'] == True]['Is Reply'].count(),
            'quote_count': group[group['Quote'] == True]['Quote'].count()
            }

In [5]:
def get_tweets(categories, limit):
    """
    :param categories: categories which you would like to compare
    :param limit: limit is the number of tweets to get per category with hash or no hash
    :return:
    """
    new_columns = ['Category', 'Is Hashtag', 'Text', 'Quote', 'Retweet Count', 'Favorite Count', 'Is reply']
    data = list()
    for category in categories:

        tweets = api.search(category, lang='en', count=limit)
        
        for tweet in tweets["statuses"]: # iterate tweets without hash
            # add the details for each tweet without hash in this category
            text = tweet["text"]
            quote = tweet["is_quote_status"]
            retweet_count = tweet["retweet_count"]
            favorite_count = tweet["favorite_count"]
            is_reply = False if tweet["in_reply_to_status_id"] == None else True
            row = (category, False, text, quote, retweet_count, favorite_count, is_reply)
            data.append(row)
            
        tweets = api.search(f"#{category}", lang='en', count=limit)
        # Using # instead of @
        for tweet in tweets["statuses"]: # iterate tweets with hashtag
            # add the details for each tweet with hash in this category
            text = tweet["text"]
            quote = tweet["is_quote_status"]
            retweet_count = tweet["retweet_count"]
            favorite_count = tweet["favorite_count"]
            is_reply = False if tweet["in_reply_to_status_id"] == None else True
            row = (category, True, text, quote, retweet_count, favorite_count, is_reply)
            data.append(row)
        print(f'processed {category}') # show progress medf.to_csv("sample_2.csv", quoting=csv.QUOTE_ALL)

    df = pd.DataFrame(data, columns=new_columns)
    return df

In [6]:
def compare_tweets_by_categories(df):
    """
    :param df: tweets dataframe
    :return: list of dictionaries of categories and hashvalues, along with summary
    """
    # convert all text to lower case
    # Remove all special characters except text and white space characters
    df['Text'] = df['Text'].str.lower().str.replace('[^\w\s]', '')
    # Remove rt(retweet) and category as it will be contained in many tweets
    df['Text'] = df['Text'].str.replace(r'[_|rt]+', '')

    result = df.groupby(['Category', 'Is Hashtag'])
    result_list = list()
    for name, group in result:
        result_list.append(
            {
                "category": name[0],
                "is_hashtag": name[1],
                "summary": get_group_summary(group, name[0])
            }
        )
    return result_list


In [7]:
# entertainment, sports, art and design, government, politics
    # health
    # research
categories = ['health','entertainment','sports','arts and design','government','politics','research']

# collect http://localhost:8888/notebooks/Twitter_Trends_Project3/twitter_charts%20(7).ipynb#tweets with categories and append to sample_1.csv
df = get_tweets(categories, 5)

# calls the method, takes two params (list of categories and number of tweets to get per category)

processed health
processed entertainment
processed sports
processed arts and design
processed government
processed politics
processed research


In [14]:
df.head(200)

,Category,Is Hashtag,Text,Quote,Retweet Count,Favorite Count,Is reply
0,health,False,Make sure you do your own research or know tha...,False,0,0,False
1,health,False,RT @KeithMcCallin: @davidsirota And wander Twi...,False,1,0,False
2,health,False,We're hiring! Click to apply: Clinical Nurse B...,False,0,0,False
3,health,False,RT @robreiner: We have a President who aligns ...,False,4775,0,False
4,health,True,RT @DrDinaKulik: Symptoms of Early Pregnancy ....,False,1,0,False
5,health,True,"The goodness of garlic, ginger, honey, palmoil...",False,0,0,False
6,health,True,RT@AMZ0NE The SECRETS of heart disease uncover...,False,0,0,False
7,health,True,RT @MHummingsPsyD: Former NFL Players With Neu...,False,1,0,False
8,health,True,Unravel the strong hold that FOOD and EMOTIONS...,False,0,0,False
9,entertainment,False,RT @realDonaldTrump: So surprised to see Conse...,False,41135,0,False


In [15]:
# Using plotly for basic-bar
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [18]:

import plotly.plotly as py
import plotly.graph_objs as go

# comparing Category with Retweet Count and forming a basic-bar using plotly
data = [go.Bar(
            x= df['Category'],
            y=df['Retweet Count']
    )]
#ploting plotly
py.iplot(data, filename='basic-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~SairaJahangir/0 or inside your plot.ly account where it is named 'basic-bar'


In [21]:
import plotly.plotly as py
import plotly.graph_objs as go

#using plotly to compare Category/Retweet Count and Category/Favorite Count

# trace1 comparing Category with Retweet Count 
trace1 = go.Bar(
    x= df['Category'],
    y=df['Retweet Count']
   
)

# trace 2 comapring Category with Favorite Count
trace2 = go.Bar(
    x=['Category'],
    y=['Favorite Count'],
   
)
#Layout trace 1 and trace 2 into one gro
data = [trace1, trace2]
layout = go.Layout(
    barmode='group'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~SairaJahangir/0 or inside your plot.ly account where it is named 'basic-bar'
